In [ ]:
install.packages("text2vec")
library(text2vec)
library(data.table)
library(glmnet)
rm(list = ls())
library(plyr)
library(ggplot2)
library(tm)
library(lsa)
datacsv = read.csv("train_talk.csv")
datacsv <- as.data.frame(datacsv)
datacsv <- datacsv[,1:3]
datacsv <- na.omit(datacsv)
setDT(datacsv)
set.seed(2016L)
all_ids <- c(1:nrow(datacsv))
datacsv$id <- all_ids
train_ids <- sample(all_ids,0.8*nrow(datacsv))
test_ids <- setdiff(all_ids,train_ids)
train <- datacsv[datacsv$id%in%train_ids,]
nrow(train)
test <- datacsv[datacsv$id%in%test_ids,]
nrow(test)
prep_fun = tolower
tok_fun = word_tokenizer
train$detail <- as.character(train$detail)
train$title <- as.character(train$title)
test$detail <- as.character(test$detail)
test$title <- as.character(test$title)
it_train = itoken(train$detail,
                  preprocessor = prep_fun,
                  tokenizer = tok_fun,
                  ids = train$id,
                  progressbar = FALSE)
it_test = test$detail %>% 
  prep_fun %>% 
  tok_fun %>% 
  itoken(ids = test$id, 
         # turn off progressbar because it won't look nice in rmd
         progressbar = FALSE)
vocab = create_vocabulary(it_train)
vectorizer = vocab_vectorizer(vocab)
t1 = Sys.time()
dtm_train = create_dtm(it_train, vectorizer)
print(difftime(Sys.time(), t1, units = 'sec'))
# define tfidf model
tfidf = TfIdf$new()
# fit model to train data and transform train data with fitted model
dtm_train_tfidf = fit_transform(dtm_train, tfidf)
# tfidf modified by fit_transform() call!
# apply pre-trained tf-idf transformation to test data
dtm_test_tfidf  = create_dtm(it_test, vectorizer) %>% 
  transform(tfidf)
str(dtm_train_tfidf)
str(train)
tr2 <- train
tr2$title <- as.factor(as.character(tr2$title))
unique(tr2$title)
str(tr2)
glmnet_classifier = cv.glmnet(x = dtm_train_tfidf, y = train[['title']], 
                              family = 'binomial', 
                              alpha = 1,
                              type.measure = "auc",
                              nfolds = NFOLDS,
                              thresh = 1e-3,
                              maxit = 1e3)

#-----------#####
